# Assignment 3: Tool-Enhanced Pricing Agent with Optimizer

## Objective
Add **external tool capabilities** to our pricing agent, allowing it to call specialized pricing optimization functions for more accurate and deterministic calculations.

## Requirements
**Tools Added:**
- Margin Calculator Tool
- Elasticity Adjustment Tool
- Competitor Matching Tool

**Agent Flow:**
1. Agent interprets inputs
2. Calls appropriate tools
3. Computes recommended price
4. Returns final explanation + number

**Tool Example:**
```python
optimizer.calculate_price(
    cost_price=400,
    target_margin=0.25,
    competitor_price=579,
    elasticity="medium"
)
```

## Setup & Dependencies

Install required packages for tool-enhanced agent implementation.

In [1]:
# Install required packages for tool-enhanced agent
!pip install -q langchain langchain-groq langchain-community
!pip install -q pandas numpy scipy
!pip install -q pydantic

In [2]:
# Import required libraries
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, ToolMessage
from langchain_core.tools import tool
import os
import getpass
import pandas as pd
import json
import math
from typing import List, Dict, Optional, Union
from pydantic import BaseModel

/Users/ashwinrs/go/agentic-ai/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Set up your Groq API key
print("Please enter your Groq API key:")
print("(You can get one free at: https://console.groq.com/)")
groq_api_key = getpass.getpass("Groq API Key: ")
os.environ["GROQ_API_KEY"] = groq_api_key
print("API key set successfully!")

Please enter your Groq API key:
(You can get one free at: https://console.groq.com/)
API key set successfully!


## Pricing Optimization Tools

Create the three external tools our agent will use for pricing calculations.

In [ ]:
# TODO: Create the three pricing tools using @tool decorator

@tool
def margin_calculator(cost_price: float, target_margin: float) -> dict:
    """
    Calculate optimal selling price based on cost and target margin.
    
    Args:
        cost_price (float): The cost to produce/acquire the product
        target_margin (float): Desired margin as decimal (e.g., 0.25 for 25%)
    
    Returns:
        dict: Contains selling_price, margin_dollar, and validation info
    """
    # TODO: Implement margin calculation logic
    validation_info = "No errors in margin"
    if target_margin >= 1.0:
        target_margin = 0.99
        validation_info = "Margin was greater or equal to 1, adjusted to 0.99"
    elif target_margin < 0.0:
        target_margin = 0.0
        validation_info = "Margin was lower than 0, adjusted to 0.0"
    selling_price = cost_price / (1 - target_margin)
    margin_dollar = selling_price - cost_price

    return {"selling_price": selling_price, "margin_dollar": margin_dollar, "validation_info": validation_info}

@tool
def elasticity_adjustment(base_price: float, elasticity: str, price_change_percent: float) -> dict:
    """
    Adjust pricing based on demand elasticity to optimize revenue.
    
    Args:
        base_price (float): Starting price point
        elasticity (str): Elasticity level - 'low', 'medium', 'medium-high', or 'high'
        price_change_percent (float): Proposed price change as percentage (e.g., 10 for 10% increase)
    
    Returns:
        dict: Contains adjusted_price, demand_impact, revenue_impact
    """
    # TODO: Implement elasticity calculation
    # HINT: Define elasticity coefficients for each level
    # Calculate demand change: elasticity_coefficient * price_change_percent
    # Calculate revenue impact: (1 + price_change) * (1 + demand_change) - 1
    
    elasticity_coefficients = {
        # TODO: Add elasticity coefficients
        "low" : -0.5,
        "medium": -1.0,
        "medium-high": -1.5,
        "high": -2.0,
    }
    
    # Validate elasticity input
    if elasticity not in elasticity_coefficients:
        return {"error": "Invalid elasticity level provided. Choose from 'low', 'medium', 'medium-high', or 'high'."}

    # calculate percentage changes as ratios
    price_change_ratio = 1 + (price_change_percent / 100)

    # Calculate adjusted price
    adjusted_price = base_price * price_change_ratio

    # Calculate demand change
    demand_change_percent = elasticity_coefficients[elasticity] * price_change_percent
    demand_change_ratio = 1 + (demand_change_percent / 100)
    demand_impact = demand_change_ratio - 1

    revenue_impact = (price_change_ratio * demand_change_ratio) -1
    return {
        "adjusted_price": adjusted_price,
        "demand_impact_percent": demand_change_percent,
        "revenue_impact_percent": revenue_impact * 100
    }
    

@tool
def competitor_matching(our_cost: float, competitor_price: float, positioning: str = "match") -> dict:
    """
    Calculate optimal pricing relative to competitor prices.
    
    Args:
        our_cost (float): Our cost to produce the product
        competitor_price (float): Competitor's selling price
        positioning (str): Strategy - 'undercut', 'match', 'premium', or 'aggressive'
    
    Returns:
        dict: Contains recommended_price, our_margin, competitive_analysis
    """
    # TODO: Implement competitive pricing logic
    # HINT: Define positioning multipliers
    # 'aggressive': 0.85, 'undercut': 0.92, 'match': 0.98, 'premium': 1.10
    # Check if recommended price covers costs
    
    positioning_strategies = {
        "aggressive": 0.85,
        "undercut": 0.92,
        "match": 0.98,
        "premium": 1.10
    }

    
    if positioning not in positioning_strategies:
        return {"error": "Invalid positioning strategy. Choose from 'undercut', 'match', 'premium', or 'aggressive'."}

    multiplier = positioning_strategies[positioning]
    recommended_price = competitor_price * multiplier
    our_margin = recommended_price - our_cost

    competitive_analysis = f"Pricing set to {positioning} competitor at ${competitor_price}. Recommended price: ${recommended_price:.2f}. Our margin: ${our_margin:.2f}."

    if recommended_price < our_cost:
        competitive_analysis += " WARNING: Recommended price is below our cost!"

    return {
        "recommended_price": recommended_price,
        "our_margin": our_margin,
        "competitive_analysis": competitive_analysis
    }


# Test the tools
print("Testing Pricing Tools:")
print("="*40)

margin_result = margin_calculator.invoke({"cost_price": 400, "target_margin": 0.25})
print(f"Margin test: {margin_result}")

elasticity_result = elasticity_adjustment.invoke({"base_price": 533, "elasticity": "medium", "price_change_percent": 8.5})
print(f"Elasticity test: {elasticity_result}")

competitor_result = competitor_matching.invoke({"our_cost": 400, "competitor_price": 579, "positioning": "match"})
print(f"Competitor test: {competitor_result}")

print("Tools implementation complete!")

Testing Pricing Tools:
Margin test: {'selling_price': 533.3333333333334, 'margin_dollar': 133.33333333333337, 'validation_info': 'No errors in margin'}
Elasticity test: {'adjusted_price': 578.305, 'demand_impact_percent': -8.5, 'revenue_impact_percent': -0.7225000000000037}
Competitor test: {'recommended_price': 567.42, 'our_margin': 167.41999999999996, 'competitive_analysis': 'Pricing set to match competitor at $579.0. Recommended price: $567.42. Our margin: $167.42.'}
Tools implementation pending...


## Tool-Enhanced Pricing Agent

Create our enhanced pricing agent that can call these external tools.

In [11]:
class ToolEnhancedPricingAgent:
    def __init__(self):
        self.llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.1, max_tokens=2048)
        
        self.tools = [margin_calculator, elasticity_adjustment, competitor_matching]
        
        self.llm_with_tools = self.llm.bind_tools(self.tools)
        
        self.system_message = SystemMessage(content="""
        You are a retail pricing agent. Follow these rules when providing a response:                                    
        # 1. Available tools: margin_calculator, elasticity_adjustment, competitor_matching
        # 2. Process: Analyze request → Call tools → Interpret results → Recommend price
        # 3. Always use tools for calculations rather than manual estimates
        """)
        
    def calculate_price(self, cost_price: float, target_margin: float = None, 
                       competitor_price: float = None, elasticity: str = None,
                       positioning: str = "match"):
        """
        Calculate optimal price using available tools
        """
        
        request_parts = [
            f"Calculate optimal pricing for a product with cost ${cost_price}"
        ]
        
        if target_margin: request_parts.append(f" Target margin: {target_margin*100:.1f}%")
        if competitor_price: request_parts.append(f" Competitor price: ${competitor_price}")
        if elasticity: request_parts.append(f" Elasticity: {elasticity}")
        if positioning: request_parts.append(f" Positioning: {positioning}")
        
        request_parts.append("\nUse your tools to calculate the optimal price and provide detailed analysis.")
        
        human_message = HumanMessage(content="\n".join(request_parts))
        messages = [self.system_message, human_message]
        response = self.llm_with_tools.invoke(messages)
        messages.append(response)
        
        while response.tool_calls:
            for tool_call in response.tool_calls:
                # Find and execute the tool
                # Add tool result to messages
                tool_name = tool_call["name"]
                tool_args = tool_call["args"]
                print(f"   Calling {tool_name} with args: {tool_args}")
                match tool_name:
                    case "margin_calculator":
                        result = margin_calculator.invoke(tool_args)
                    case "elasticity_adjustment":
                        result = elasticity_adjustment.invoke(tool_args)
                    case "competitor_matching":
                        result = competitor_matching.invoke(tool_args)
                    case _:
                        result = "Tool NotImplemented Error"
                print(f"   Tool Result: {result}")

                messages.append(ToolMessage(content=str(result),
                                            tool_call_id=tool_call["id"]))
            response = self.llm_with_tools.invoke(messages)
            if not response.tool_calls:
                return response.content
            messages.append(response)
        
        final_response = self.llm_with_tools.invoke(messages)
        return final_response.content

# Initialize the tool-enhanced agent
print("Initializing Tool-Enhanced Pricing Agent...")
optimizer = ToolEnhancedPricingAgent()
print(f"Available tools: {[tool.name for tool in optimizer.tools]}")

Initializing Tool-Enhanced Pricing Agent...
Available tools: ['margin_calculator', 'elasticity_adjustment', 'competitor_matching']


## Testing the Tool-Enhanced Agent

Test our agent with the assignment example and various scenarios.

In [12]:
# Test with the assignment example
print("Assignment Example: Tool-Enhanced Pricing")
print("="*50)

result = optimizer.calculate_price(
    cost_price=400,
    target_margin=0.25,
    competitor_price=579, 
    elasticity="medium"
)

print(result)


Assignment Example: Tool-Enhanced Pricing
   Calling margin_calculator with args: {'cost_price': 400, 'target_margin': 0.25}
   Tool Result: {'selling_price': 533.3333333333334, 'margin_dollar': 133.33333333333337, 'validation_info': 'No errors in margin'}
   Calling competitor_matching with args: {'competitor_price': 579, 'our_cost': 400, 'positioning': 'match'}
   Tool Result: {'recommended_price': 567.42, 'our_margin': 167.41999999999996, 'competitive_analysis': 'Pricing set to match competitor at $579.0. Recommended price: $567.42. Our margin: $167.42.'}
   Calling elasticity_adjustment with args: {'base_price': 500, 'elasticity': 'medium', 'price_change_percent': 0}
   Tool Result: {'adjusted_price': 500.0, 'demand_impact_percent': -0.0, 'revenue_impact_percent': 0.0}
Based on the analysis, the optimal price for the product is $533.33, which provides a 25% margin. However, since the competitor's price is $579, we need to adjust our price to match the competitor's price. Therefore,

In [ ]:
# TODO: Create comprehensive analysis method
def get_comprehensive_analysis(product_name: str, category: str, 
                             cost_price: float, target_margin: float = None,
                             competitor_price: float = None, elasticity: str = None) -> str:
    """
    Get comprehensive pricing analysis using multiple tools
    """
    
    analysis_report = f"Comprehensive Pricing Analysis for {product_name} ({category}):\n\n"
    analysis_report += f"Cost Price: ${cost_price:.2f}\n"

    recommended_price = None

    # 1. Use margin_calculator if target margin provided
    if target_margin is not None:
        margin_result = margin_calculator.invoke({"cost_price": cost_price, "target_margin": target_margin})
        analysis_report += f"-- Margin-Based Pricing --\n"
        analysis_report += f"Target Margin ({target_margin*100:.1f}%): Selling Price = ${margin_result['selling_price']:.2f}, Margin Dollar = ${margin_result['margin_dollar']:.2f}. ({margin_result['validation_info']})\n"
        if recommended_price is None:
            recommended_price = margin_result['selling_price']

    # 2. Use competitor_matching if competitor price available
    if competitor_price is not None:
        analysis_report += f"\n-- Competitive Pricing --\n"
        for positioning_strategy in ['aggressive', 'undercut', 'match', 'premium']:
            comp_result = competitor_matching.invoke({"our_cost": cost_price, "competitor_price": competitor_price, "positioning": positioning_strategy})
            if "error" not in comp_result:
                analysis_report += f"Strategy '{positioning_strategy}': {comp_result['competitive_analysis']}\n"
                if recommended_price is None and positioning_strategy == 'match': # Default to match if no other recommendation
                    recommended_price = comp_result['recommended_price']
            else:
                analysis_report += f"Strategy '{positioning_strategy}': Error - {comp_result['error']}\n"

    # 3. Use elasticity_adjustment to test price sensitivity
    if elasticity is not None and recommended_price is not None:
        analysis_report += f"\n-- Elasticity-Based Impact Analysis (based on recommended price: ${recommended_price:.2f}) --\n"
        test_price_changes = [-5, 0, 5, 10] # Test -5%, 0%, 5%, 10% price changes
        for change in test_price_changes:
            elasticity_result = elasticity_adjustment.invoke({"base_price": recommended_price, "elasticity": elasticity, "price_change_percent": float(change)})
            if "error" not in elasticity_result:
                analysis_report += f"  Price Change {change}% (new price ${elasticity_result['adjusted_price']:.2f}): Demand Impact = {elasticity_result['demand_impact_percent']:.2f}%, Revenue Impact = {elasticity_result['revenue_impact_percent']:.2f}%\n"
            else:
                analysis_report += f"  Price Change {change}%: Error - {elasticity_result['error']}\n"

    # 4. Provide final recommendation with reasoning
    analysis_report += f"\n-- Final Recommendation --\n"
    if recommended_price is not None:
        analysis_report += f"Based on the analysis, a recommended base selling price for {product_name} is ${recommended_price:.2f}.\n"
        if target_margin is not None and recommended_price < cost_price / (1 - target_margin):
            analysis_report += f"This price might be below the target margin of {target_margin*100:.1f}% if other factors (like competitor pricing) were prioritized.\n"
        if competitor_price is not None:
             analysis_report += f"Consider competitive landscape and demand elasticity when making final adjustments.\n"

    else:
        analysis_report += f"Unable to provide a specific price recommendation due to insufficient information.\n"

    return analysis_report

# Test comprehensive analysis
print("Comprehensive Analysis: Puma Sneakers")
print("="*45)

comprehensive_result = get_comprehensive_analysis(
    product_name="Puma Training Sneakers",
    category="Footwear",
    cost_price=400,
    target_margin=0.25,
    competitor_price=579,
    elasticity="medium"
)

print(comprehensive_result)

print("Implementation pending...")

## Testing Different Scenarios

Test various pricing scenarios to see how tools enhance decision-making.

In [ ]:
# TODO: Scenario 1: High elasticity product (Electronics)
print("Scenario 1: High Elasticity Electronics")
print("="*40)

# TODO: Test with electronics parameters
electronics_result = get_comprehensive_analysis(
    product_name="Gaming Laptop",
    category="Electronics", 
    cost_price=750,
    target_margin=0.18,
    competitor_price=950,
    elasticity="high"
)

print(electronics_result)

In [ ]:
# TODO: Scenario 2: Luxury goods (Low elasticity)
print("Scenario 2: Low Elasticity Luxury Goods")
print("="*42)

# TODO: Test with luxury goods parameters
luxury_result = get_comprehensive_analysis(
    product_name="Designer Watch",
    category="Luxury Goods",
    cost_price=300,
    target_margin=0.60,
    competitor_price=850,
    elasticity="low"
)

print(luxury_result)

In [ ]:
# TODO: Scenario 3: Aggressive competitive pricing
print("Scenario 3: Aggressive Market Entry")
print("="*35)

# TODO: Test aggressive pricing strategy
# aggressive_result = optimizer.calculate_price(
#     cost_price=60,
#     competitor_price=95,
#     positioning="aggressive",
#     elasticity="medium-high"
# )

# print(aggressive_result)

print("Aggressive pricing testing pending...")

## Tool Performance Analysis

Analyze how tools improve pricing accuracy and decision-making.

In [ ]:
def analyze_tool_effectiveness():
    """Compare tool-enhanced vs manual calculation approaches"""
    
    print("Tool Effectiveness Analysis")
    print("="*30)
    
    # TODO: Create test cases comparing manual estimates vs tool results
    test_cases = [
        {
            "name": "Standard Margin Calculation",
            "cost": 100,
            "target_margin": 0.30,
            "manual_estimate": 130,  # Common error: adding 30% instead of dividing
            "tool_function": "margin_calculator"
        },
        # TODO: Add more test cases for elasticity and competitive pricing
    ]
    
    # TODO: Compare manual estimates with tool results
    for i, case in enumerate(test_cases, 1):
        print(f"\nTest Case {i}: {case['name']}")
        print("-" * 30)
        
        # TODO: Show manual vs tool comparison
        print(f"Manual estimate: ${case['manual_estimate']}")
        # tool_result = call_appropriate_tool(case)
        # print(f"Tool result: {tool_result}")
        print("Tool comparison pending...")

# Run analysis
analyze_tool_effectiveness()

## Interactive Pricing Optimizer

Create an interactive tool to test different pricing scenarios.

In [ ]:
def interactive_pricing_optimizer():
    """
    Interactive pricing consultation with tool-enhanced agent
    """
    
    print("Interactive Tool-Enhanced Pricing Optimizer")
    print("="*45)
    print("Available elasticity levels: low, medium, medium-high, high")
    print("Positioning strategies: aggressive, undercut, match, premium")
    print()
    
    # TODO: Collect inputs from user
    # product = input("Product name: ")
    # category = input("Category: ")
    # cost = float(input("Cost price ($): "))
    # etc.
    
    print("Interactive mode commented out - uncomment to use")
    
    # TODO: Get comprehensive analysis with collected inputs
    # analysis = get_comprehensive_analysis(...)
    # print(analysis)

# Uncomment to run interactive optimizer
# interactive_pricing_optimizer()

## Tool Comparison Study

Compare different pricing approaches using our tools.

In [ ]:
def pricing_strategy_comparison(cost: float, target_margin: float, competitor: float, elasticity: str):
    """
    Compare different pricing strategies using tools
    """
    
    print(f"Pricing Strategy Comparison")
    print(f"Cost: ${cost}, Target Margin: {target_margin*100:.0f}%, Competitor: ${competitor}, Elasticity: {elasticity}")
    print("="*80)
    
    # TODO: Compare different strategies
    # Strategy 1: Margin-based pricing
    # margin_result = margin_calculator.invoke(...)
    
    # Strategy 2: Competitive pricing strategies
    # for strategy in ['aggressive', 'undercut', 'match', 'premium']:
    #     comp_result = competitor_matching.invoke(...)
    
    # Strategy 3: Elasticity-optimized pricing  
    # Test different price adjustments with elasticity_adjustment
    
    print("Strategy comparison implementation pending...")

# Test pricing strategy comparison
print("Example: Footwear Pricing Strategy Comparison")
pricing_strategy_comparison(
    cost=90,
    target_margin=0.35,
    competitor=160,
    elasticity="medium-high"
)

## Edge Case Testing

Test how tools handle challenging pricing scenarios.

In [ ]:
def test_edge_cases():
    """
    Test edge cases and error handling in pricing tools
    """
    
    print("Edge Case Testing")
    print("="*20)
    
    # TODO: Test edge cases
    # Edge Case 1: Impossible margin (90%)
    # edge1 = margin_calculator.invoke({"cost_price": 100, "target_margin": 0.90})
    
    # Edge Case 2: Invalid elasticity
    # edge2 = elasticity_adjustment.invoke({"base_price": 100, "elasticity": "super-high", "price_change_percent": 10})
    
    # Edge Case 3: Competitor price below cost
    # edge3 = competitor_matching.invoke({"our_cost": 200, "competitor_price": 150, "positioning": "match"})
    
    # Edge Case 4: Extreme price elasticity impact
    # edge4 = elasticity_adjustment.invoke({"base_price": 100, "elasticity": "high", "price_change_percent": 50})
    
    print("Edge case testing implementation pending...")

# Run edge case tests
test_edge_cases()

## Your Turn: Implement the Tools and Agent

**Step 1:** Implement the three pricing tools:
- `margin_calculator`: Calculate price from cost and margin
- `elasticity_adjustment`: Calculate demand/revenue impact of price changes
- `competitor_matching`: Calculate competitive pricing strategies

**Step 2:** Implement the `ToolEnhancedPricingAgent`:
- Initialize LLM with tool binding
- Create tool calling logic
- Process tool results and generate recommendations

**Step 3:** Test with the assignment example:
```python
optimizer.calculate_price(
    cost_price=400,
    target_margin=0.25,
    competitor_price=579,
    elasticity="medium"
)
```

**Step 4:** Extend with additional tools and scenarios

In [ ]:
# Your implementation space

# TODO: Implement margin_calculator tool
# TODO: Implement elasticity_adjustment tool
# TODO: Implement competitor_matching tool
# TODO: Implement ToolEnhancedPricingAgent class
# TODO: Test with assignment example
# TODO: Add additional test cases

print("Ready for your implementation!")
print("Follow the TODOs above to build the tool-enhanced pricing agent.")